In [1]:
# This cell defines the function that can be used for 

# This function calculates initial condition profiles based on method described in paper
# The inputs to this function are the wall potential (required to be negative) and expected ion particle flux at the wall (must be greater than 0.5 otherwise solution may not converge)
# Algorith based on Robertson, S. (2013) PPCF: http://dx.doi.org/10.1088/0741-3335/55/9/093001 

# Output to this function is ordered as 
# 0: x
# 1: phi
# 2: ni
# 3: ne
# 4: ui

# Load libraries
import numpy as np
from  scipy.integrate  import  odeint
import math 

# Define function
def robertson_ODE_solver(phi_p, expected_niui):
    # Check to see if phi_p is greater than 0. If it is, make negative, and print that
    if phi_p > 0:
        print("phi_p has been inputted to be greater than zero. This analysis only works for a classical sheath (not a reverse sheath).")
        print("Therefore, phi_p must be negative. This function has converted the inputted positive potential into a negative one.\n")
        phi_p = -phi_p
    
    # Print the inputted values
    print("phi_p=%.10f" % (phi_p))
    print("expected_niui=%.10f" % (expected_niui))
        
    # Set the desired tolerance (the number chosen already is likely much lower than needed for any simulation)
    TOL = 1.0e-12
    
    # Define initial domain length. This is purposely set to be small so that we can arbitrarily increase L until we reach the appropriate value. 
    L = 2

    # Note that this function has not yet been tested for small phi_p. If the actual desired length is less than the initially defined length, this function might not work properly.
    # So be careful if this is the case (and this is also the reason we choose a small initial L)
    
    # Define the previous distance changed as the initial L. This becomes important in future iterations as we are trying to converge to the solution
    Ldiff = L
    
    # Set number of points per Debye length. Used to make the x grid
    num_per_lambdaD = 2
    
    # Define the direction that was previously taken
    # 0 means we previously subtracted. 
    # 1 means we previously added
    # Initialize this as 1
    direction = 1

    # Define a reverse switch to determine if we have overshot yet
    # It not, it is 0 and we will keep doubling L until we do
    # If we have overshot, then set this to 1 and we will slowly change L to converge to the correct solution
    reverseSwitch = 0
    
    # Define the ODEs that we need to solve. 
    def  robertson(y, x, S):
      phi , E, u = y
      dydx = [-E, R*x/u - np.exp(phi), E/u - u/x]
      return  dydx

    # Set a max number of iterations in case this doesn't converge or blows up
    max_iter = 500
    
    # Begin iterations
    for i in range(0,max_iter):    
        # Make the grid. 

        # The grid will have at least 2 cells per Debye length    
        # Choose the ceil function so that we end up with an integer value
        numX = math.ceil(L*num_per_lambdaD)

        # Get the dx
        dx = L / numX

        # Make the x grid using linspace
        x = np.linspace(dx, L, numX)        

        # Define R (ionization rate)
        R = expected_niui / L

        # Set the initial conditions to the ODEs
        y0 = [-R**2*dx**2, 2*R**2*dx , R*dx]

        # Solve the ODEs
        sol = odeint(robertson , y0 , x, args=(R,))

        # Extract just the potential
        phi = sol[:,0]

        # Print solution
        print('i=%d  L=%.10f  phi=%.10f  phip=%.10f' % (i,L,phi[-1],phi_p))
        
        # Save the current L to use as reference for future calculations
        prevL = L

        # Check to see if wall potential is within tolerance
        # If so, break the loop, we have obtained the desired domain length
        if phi[-1] <= phi_p+TOL and phi[-1] >= phi_p-TOL:        
            break
        elif phi[-1] >= phi_p and reverseSwitch == 0:
            # In this case, we have not yet overshot the desired wall potential
            # We will attempt to get closer to the solution by multiplying L by 2            
            L = L*2.0        
        elif phi[-1] >= phi_p and reverseSwitch == 1:
            # In this case, we have overshot at some point before but are not at the potential yet
            # Therefore, we don't want to just multiply L by 2 since we would greatly overshoot again
            # In this case, we want to add by half of the previous distance we traveled            
            L = L + 0.5*Ldiff    
        elif phi[-1] <= phi_p:
            # In this case, we have overshot the plasma potential
            # First, if the reverseSwitch has not turned on yet, we must turn it on
            reverseSwitch = 1

            # In this case, we want to decrease L by half of the previously moved distance            
            L = L - 0.5*Ldiff

        # Calculate the difference between the last two lengths
        Ldiff = np.abs(L - prevL)

        # If Ldiff is less than the tolerance, than we have converged enough in distance to the approximate solution
        # Sometimes, the solution will converge to a solution that is slightly different based on the wall potential
        # This is a check to still obtain a reasonable enough initial condition
        if Ldiff < TOL:
            break  
        
        # If L is greater than 10^6, break the loop. It is highly likely that L is blowing up
        if L > 1.e6:
            print("L > 1.e6. Solution likely blew up. Results are invalid.")
            break
            
    # Calculate other values we care about
    ui = sol[:,2]
    ne = np.exp(phi)
    ni = R * x / ui 
    E = sol[:,1]
    
    # Place final values into an array that we will output
    # Array index as x, phi, ni, ne, ui
    output_array = np.zeros((len(phi),6))
    
    output_array[:,0] = x
    output_array[:,1] = phi
    output_array[:,2] = ni
    output_array[:,3] = ne
    output_array[:,4] = ui
    output_array[:,5] = E
    
    return output_array

In [ ]:
# This cell outputs the left and right wall profiles to be used in Gkeyll input files for biased wall sims
# This cell will make use of the Robertson profile solver from the previous cell
# This cell assumes that the Poisson BCs are dirichlet with the right wall at 0 and the left wall at some defined potential
# The inputs are phiL, Ti, Te, mi, and the expected particle flux
# This cell also plots and saves the profiles

# Note: You must run the previous cell first to use this cell

# Load libraries
import numpy as np
from  scipy.integrate  import  odeint
import math 


############## Set the input values  #############################
phiL_SI = 10.e3      # V  
Ti = 2000.      # eV
Te = 2000.      # eV
mi = 1.67262192369e-27     # kg
expected_niui = 0.55    # Must be greater than 
saveName = '10'   # Name you want to use for your saved files
fileDir = '/path/here/'   # Set the directory where you want to save the data

# Set other important constants
me = 9.1093837015e-31      # kg

# Calculate the normalized potential at the left wall
# Note that because the temperature is in eV, the normalization for the potential is simply to divide by Te
phiL = phiL_SI / Te

# Calculate the floating wall potential (i.e. the plasma potential if there was no bias between the left and right walls)
# See Eq. 2.60 from textbook The Plasma Boundary of Magnetic Fusion Devices by P. Stangeby
phi_sf = -0.5*np.log(2*math.pi*me*(1+Ti/Te)/mi)

# Calculate plasma potential (including effects of bias)
# See Eq. 2.67 from textbook The Plasma Boundary of Magnetic Fusion Devices by P. Stangeby
phi_p = phiL - np.log(2*np.exp(-phi_sf)/(1+np.exp(-phiL)) )

# Calculate the delta potential for the left and right walls
deltaPhi_L = phi_p - phiL
deltaPhi_R = phi_p

# Calculate left wall profiles
leftProfiles = robertson_ODE_solver(-deltaPhi_L, expected_niui)

# We need to do some transformations to make this easier to work with for use in Gkeyll
# Need to flip all of the profiles since the wall is on the left
niL = np.flip(leftProfiles[:,2])
neL = np.flip(leftProfiles[:,3])

# Velocity needs to be multiplied by negative one since the particles should be moving to the left
uiL = -np.flip(leftProfiles[:,4])

# The potential needs to be converted to a new ground frame. Do this by subtracting by phi_p
phiL = np.flip(leftProfiles[:,1]) + phi_p

# The electric field needs to be multipled by negative one
EL = -np.flip(leftProfiles[:,5])

# The x value should remain unflipped, but we want it to start at 0 (since we flipped everything else).
# Therefore, subtract the zeroth element of the previous x to set the start point to 0
xL = leftProfiles[:,0] - leftProfiles[0,0]

# Calculate right wall profiles
rightProfiles = robertson_ODE_solver(-deltaPhi_R, expected_niui)

# The only transformation that needs to be done for the right side is for the potential. Add phi_p
xR = rightProfiles[:,0]
phiR = rightProfiles[:,1] + phi_p
niR = rightProfiles[:,2]
neR = rightProfiles[:,3]
uiR = rightProfiles[:,4]
ER = rightProfiles[:,4]

# Print out sheath entrances for each side.
# For now, sheath entrance is defined as 1% quasineutrality

# Save the data to text files
np.savetxt(fileDir + 'xL_' + saveName + '.txt',xL)
np.savetxt(fileDir + 'phiL_' + saveName + '.txt',phiL)
np.savetxt(fileDir + 'uiL_' + saveName + '.txt',uiL)
np.savetxt(fileDir + 'niL_' + saveName + '.txt',niL)
np.savetxt(fileDir + 'neL_' + saveName + '.txt',neL)
np.savetxt(fileDir + 'EL_' + saveName + '.txt',EL)

np.savetxt(fileDir + 'xR_' + saveName + '.txt',xR)
np.savetxt(fileDir + 'phiR_' + saveName + '.txt',phiR)
np.savetxt(fileDir + 'uiR_' + saveName + '.txt',uiR)
np.savetxt(fileDir + 'niR_' + saveName + '.txt',niR)
np.savetxt(fileDir + 'neR_' + saveName + '.txt',neR)
np.savetxt(fileDir + 'ER_' + saveName + '.txt',ER)